#Pneumonia Detection with Image Classification



### The architecture used is AlexNet following this [paper](https://link.springer.com/article/10.1007/s12559-020-09787-5)

### Data is taken from [Kaggle](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia).

### [Instructions](https://medium.com/analytics-vidhya/how-to-fetch-kaggle-datasets-into-google-colab-ea682569851a) on working with the Kaggle API 

#### The thresholds are by default set so that images with a pneumonia doubt are given more importance. In doing so, it may label normal people as pneumonia  

### An alternative model can be found [here](https://github.com/hasan-farooq/Neural-Network/blob/main/model.h5) for loading and predicting (MANUALLY)

### Loading the model
-> Download libraries and model

-> Import libraries




```
# Follow these steps then...
model = tf.keras.models.load_model("model.h5")
data = np.array(Image.open('your image.format(i.e.png)').resize((150,150)))
temp = Image.fromarray(data)
rgb_temp = temp.convert('RGB')
plt.imshow(rgb_temp)
final = np.array(rgb)
final = final[np.newaxis is None,:,:,:]
final = final/255
result = model.predict([[final]])
```
The result will be probabilties of yes/no



### Enabling GPU

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
%cd /content/gdrive/My Drive/Kaggle

## Libraries

In [ ]:
import re
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow import keras
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
%matplotlib inline

### Uncomment following cells to download data

In [ ]:
# !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
# !unzip \*.zip 

In [ ]:
test_path = "chest_xray/chest_xray/test"
train_path = "chest_xray/chest_xray/train"
val_path = "chest_xray/chest_xray/val"

epochs = 10
batch_size = 16
img_width, img_height = 227, 227
input_shape = (img_width, img_height, 3)

### Uncomment the following cell to see if image is loaded 

In [12]:
# normal_image = load_img(test_path+"/NORMAL/NORMAL2-IM-0041-0001.jpeg",color_mode = 'grayscale', target_size = (128,128))
# plt.imshow(normal_image)

## AlexNet Architecture 
![picture](https://anhreynolds.com/img/alexnet.png)

In [ ]:

class CNN_Model():

  def __init__(self,input_shape = (227,227,3), optimizer = 'adam', batch_size = 16, epochs=10):
    self.batch_size = batch_size
    self.epochs = epochs
    self.optimizer = optimizer
    self.loss = "binary_crossentropy"
    self.metric = "accuracy"

    model = Sequential()
    
    model.add(Conv2D(filters=(96),kernel_size=(11,11),strides=(4,4),input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    # model.add(MaxPooling2D(pool_size=(2,2))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=(256),kernel_size=(5,5),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
    model.add(BatchNormalization())

    model.add(Conv2D(filters=(384),kernel_size=(3,3),padding='same'))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=(384),kernel_size=(3,3)))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=(256),kernel_size=(3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    self.model = model

  def prepare(self):
    self.model.compile(
        loss=self.loss,
        optimizer=self.optimizer,
        metrics=[self.metric]
        )
  
  def train(self, data, data_size, val=None, val_size=None,):
    self.model.fit(
      data,
      steps_per_epoch = data_size // self.batch_size,
      epochs=self.epochs,
      validation_data = val,
      validation_steps = val_size // self.batch_size
    )
  
  def evaluate(self, test_data):
    scores = self.model.evaluate(test_data)
    print(self.metric + " -> " + str(scores))
    return scores

  def describe(self):
    return self.model.summary()

  def predict(self, image_path):
    data = np.array(Image.open(image).resize((150,150)))
    temp = Image.fromarray(data)
    temp_rgb = temp.convert('RGB')
    plt.imshow(temp_rgb)
    final = np.array(rgb)
    final = final[np.newaxis is None,:,:,:]
    final = final/255
    result = self.model.predict([[final]])
    return result

  def save_model(self,filename="model.h5"):
    self.model.save(filename)

  def plot(self,X,y, threshold=0.65):
    predictions = self.model.predict(X)
    # threshold = 0.65
    fig=plt.figure(figsize=(15,15))
    columns = 4
    rows = 4

    for i in range(1,16):
        flag = y[i-1]
        if (flag < 1):
          result = " Real : No"
        else:
          result = " Real : Yes"
        img = X[i-1]
        temp = fig.add_subplot(rows, columns, i)
        if predictions[i-1] > threshold:
          flag
          temp.title.set_text("Model : Yes |" + result)
        else:
          temp.title.set_text("Model : No |" + result)
        plt.imshow(img)

    fig.suptitle('Dangerous (Pneumonia)', fontsize=15)
    plt.show()





## Processing Data 

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
validation_generator = test_datagen.flow_from_directory(
    val_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

## Initializing Model

In [ ]:
model = CNN_Model()

In [ ]:
model.prepare()

In [ ]:
model.train(train_generator,data_size=5217,val=validation_generator,val_size=17)

## Plotting a batch

In [ ]:
x_batch, y_batch = (test_generator[0])

In [ ]:
model.evaluate(test_data=test_generator)

In [ ]:
model.plot(x_batch,y_batch)

Here are the [First Results](https://drive.google.com/file/d/1j86XgT2QYrCcQYZZuem2_tUazthXYUsB/view) with default parameters


### To try any image, download it and give the image file-path in predict function. 